In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170801].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('model1.csv',index=False,header=None)



import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170901].reset_index(drop=True)
print(flow.shape)

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [cc,dc]
        l.extend(price[len(price)-diff:len(price)])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('model2.csv',index=False,header=None)

import pandas as pd
file = [
    'model1.csv',  #1212
   'model2.csv' # 1306
    
       ]
sub1 = pd.read_csv(file[0],header=None)
sub2 = pd.read_csv(file[1],header=None)

sub = pd.merge(sub1,sub2,how='left',on=[0,1,2])

for i in range(3,6):
    print(sub[[str(i)+'_x',str(i)+'_y']].corr().values[0][1])
    sub[i] = sub[str(i)+'_x']*0.8+sub[str(i)+'_y']*0.2
sub[[0,1,2,3,4,5]].to_csv('model_ronghe.csv',header=None,index=False)

(26852, 6)
(20874, 6)
[44.69751229226826, 85.52046859588376, 88.40208277154342]
72.8733545532318
20180302
(20972, 6)


[44.436189198866145, 83.95356550114914, 86.34951052055303]
71.57975507352278
20180303
(21070, 6)


[43.28489676591276, 85.10010144379773, 87.69912379290403]
72.02804066753818
20180304
(21168, 6)


[44.03714596853278, 84.32026597022043, 86.89150513207815]
71.74963902361046
20180305
(21266, 6)


[43.643955420228494, 85.51398541438121, 88.74355635409503]
72.63383239623491
20180306
(21364, 6)


[43.24747944110648, 84.8308969628551, 86.30285613687104]
71.46041084694421
20180307
(21462, 6)


[43.028022218699434, 84.38232550219519, 86.38083477727515]
71.26372749938993
20180308
(21560, 6)


[42.67594978353502, 83.80482090029884, 85.88846564997682]
70.78974544460355
20180309
(21658, 6)


[42.35489570286889, 82.7612563147383, 85.44982538890545]
70.18865913550421
20180310
(21756, 6)


[42.580926025490264, 83.76197550690145, 84.3052508583198]
70.21605079690384
20180311
(21854, 6)


In [2]:
sub

0                                 1  \
0     20180302  06d86ef037e4bd311b94467c3320ff38   
1     20180302  ee2ff207184bf16b4a0aec0f97900c27   
2     20180302  06d86ef037e4bd311b94467c3320ff38   
3     20180302  3f7f0ce35d6d0a08377eb2efe2189f4f   
4     20180302  06d86ef037e4bd311b94467c3320ff38   
5     20180302  a20d041605db832309e26c003c626719   
6     20180302  a20d041605db832309e26c003c626719   
7     20180302  a20d041605db832309e26c003c626719   
8     20180302  a20d041605db832309e26c003c626719   
9     20180302  a20d041605db832309e26c003c626719   
10    20180302  a20d041605db832309e26c003c626719   
11    20180302  a20d041605db832309e26c003c626719   
12    20180302  a20d041605db832309e26c003c626719   
13    20180302  a20d041605db832309e26c003c626719   
14    20180302  a20d041605db832309e26c003c626719   
15    20180302  a20d041605db832309e26c003c626719   
16    20180302  a20d041605db832309e26c003c626719   
17    20180302  a20d041605db832309e26c003c626719   
18    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
19    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
20    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
21    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
22    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
23    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
24    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
25    20180302  a20d041605db832309e26c003c626719   
26    20180302  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
27    20180302  a20d041605db832309e26c003c626719   
28    20180302  3f7f0ce35d6d0a08377eb2efe2189f4f   
29    20180302  06d86ef037e4bd311b94467c3320ff38   
...        ...                               ...   
1440  20180316  06d86ef037e4bd311b94467c3320ff38   
1441  20180316  06d86ef037e4bd311b94467c3320ff38   
1442  20180316  06d86ef037e4bd311b94467c3320ff38   
1443  20180316  5615dc7c1af1f7dabd80bd8b8ecb1ea0   
1444  20180316  06d86ef037e4bd311b94467c3320ff38   
1445  20180316  c7537db4101856877ea6381d0174283c   
1446  20180316  c7537db4101856877ea6381d0174283c   
1447  20180316  c7537db4101856877ea6381d0174283c   
1448  20180316  c7537db4101856877ea6381d0174283c   
1449  20180316  58a33c947775af5de36841c9f553317d   
1450  20180316  c7537db4101856877ea6381d0174283c   
1451  20180316  58a33c947775af5de36841c9f553317d   
1452  20180316  58a33c947775af5de36841c9f553317d   
1453  20180316  58a33c947775af5de36841c9f553317d   
1454  20180316  58a33c947775af5de36841c9f553317d   
1455  20180316  58a33c947775af5de36841c9f553317d   
1456  20180316  58a33c947775af5de36841c9f553317d   
1457  20180316  58a33c947775af5de36841c9f553317d   
1458  20180316  58a33c947775af5de36841c9f553317d   
1459  20180316  58a33c947775af5de36841c9f553317d   
1460  20180316  58a33c947775af5de36841c9f553317d   
1461  20180316  c7537db4101856877ea6381d0174283c   
1462  20180316  58a33c947775af5de36841c9f553317d   
1463  20180316  58a33c947775af5de36841c9f553317d   
1464  20180316  58a33c947775af5de36841c9f553317d   
1465  20180316  c7537db4101856877ea6381d0174283c   
1466  20180316  c7537db4101856877ea6381d0174283c   
1467  20180316  c7537db4101856877ea6381d0174283c   
1468  20180316  58a33c947775af5de36841c9f553317d   
1469  20180316  c7537db4101856877ea6381d0174283c   

                                     2          3_x          4_x          5_x  \
0     a07711a54a122bee09726fc7489f2c1b    76.791233    41.838064    43.056886   
1     611597091d964406138868766d513d10   291.425297   215.823954   210.294897   
2     ccbf7a1a33c7c6926366ba5f6e2f1e83   124.408545    91.526251    87.487111   
3     8e45a723ac5d47626151b4f0759d343e    61.972167    58.259677    52.467020   
4     85792b2278de59316d1158f6a97537ec   145.315117    75.867427    75.519675   
5     879c99a1536ce81df8e84c0c9cf6ff68  1663.354334  2935.479871  2960.105591   
6     eed0d29bb4d08163ddd02c629a51cfb7   327.018285   636.063212   644.403072   
7     ac12452bc2dff4b1d376ed517b9f74f4  1980.005933  3087.994083  3058.996239   
8     6ebca9d3153de7a6bbad61456d7071c8   247.905480   303.8